# Tabla de Contenido
* [1. Preparación del Notebook](#prework)
* [2. Descripición de la Misión ](#mision)
    * [2.1 Propiedades del Sistema](#propiedades)
    * [2.2 Estado del Sistema](#estado)
    * [2.3 Perfil de la misión](#perfil)
* [3. Modelo del Ambiente](#ambiente)
    * [3.1 Atmosférico](#atmosferico)
    * [3.2 Termico](#termico)
    * [3.3 Tierra](#tierra)
* [4. Modelo de Trayectoria](#trayectoria)
    * [4.1 Viento](#atmosferico)
    * [4.2 Geométrico](#termico)
    * [4.3 Dinámico](#tierra)


<!---

* [5. Resultados y Discusión](#trayectoria)
    * [5.1 Graficas](#atmosferico)
    * [5.2 Metricas](#termico)
    * [5.3 No sé jeje](#tierra)

-->






# 1. Preparación del Notebook  <a name="prework"></a>

In [1]:
%matplotlib inline                 
import matplotlib.pyplot as plt   # Generación de gráficos a partir de listas o arrays. 
import numpy as np                # Manipulación de vectores, matrix y tensores

> ⚠️**Warning:** Do not push the big red button.

> 📝 **Note:** Sunrises are beautiful.

> 💡💡 **Tip:** Remember to appreciate the little things in life.

## Información de sesión

Como una buena práctica colocamos la información de sesión al final de este notebook realizado. De esta forma, otras personas podrán ver qué versiones de librerías y dependencias se utilizaron para nuestro análisis.

Además, esto permite ver si llegara alguien a tener un problema, puede compartir esta información con la comunidad o nosotros los desarrolladores de este notebook para que sea más fácil replicar el error y poder ayudar.

In [ ]:
# Lo instalamos debido a que no viene por default en los notebooks
! pip install session-info

# Importamos la librería
import session_info

# Lanzamos el comando
session_info.show()


# 2. Descripición de la Misión  <a name="mision"></a>

<figure>
    <img src = "../img/FasesDeVueloMission_StratoBalloon.png"
         alt = "Fase de Vuelo"
         name = "Fase de Vuelo"
         style="height: 400px; width:800px;">
    <figcaption> Fig. 1 Fase de Vuelo.</figcaption>
</figure>




Las operaciones con globos de gran altitud se resumen en 5 fases:
- **Fase 1:** es el lanzamiento de la sonda desde unas coordenadas predefinidas.
- **Fase 2:** es el ascenso de la sonda hasta una altura de varios kilómetros determinado por el fabricante del globo [7] y el peso de los instrumentos.
- **Fase 3:** es la explosión del globo
- **Fase 4:** es el descenso con solo el paracaídas y la carga útil.
- **Fase 5:** que es la recuperación de la sonda con ayuda de los instrumentos de navegación

## 2.1 Propiedades del Sistema  <a name="propiedades"></a>


Son las cosas que son costantes de la misión por asi decirlo

In [2]:
# Sonde    
BalloonMass = 0.5    # kg
PayloadMass = 4      # kg
TetherMass = 0.1     # kg
ParachuteMass = 0.5  # kg
rho_He0 = 0.179      # Density of Helium at sea level (kg/m3)
C_dBalloon = 0.47    # drag coefficient of balloon
C_dadded = 0.25      # c_Added coefficient

# Parachute
Cd_pchute = 0.3	# Parachute drag coefficient
Cd_spchute = 0.1	# parachute side drag coefficient
PDiam = 2		# parachute diameter (m)
PSideL = 0.1		# parachute side length, flaps (m)

## 2.2 Estado del Sistema  <a name="estado"></a>

Fases de vuelo y las condiciones en las cuales se encuentra la sonda en el espacio

In [3]:
system_stage = 0		# Mission stage:  0=balloon 1=parachute 2=gliding
v_init = 0			# velocity m/s
x_init = 0			# x,y,z position
y_init = 0
z_init = 1
balloon_diam_init = 2.3	# balloon initial diamenter
gamma_init = pi/2		# gamma angle
chi_init = 0# chi angle
lat_init = 13.808826
lon_init = -89.328988

NameError: name 'pi' is not defined

## 2.3 Perfil de la mision  <a name="perfil"></a>


Objetivos de activación de la misión

In [ ]:
balloon_separation = 30000	# m
parachute1_separation = 16000	# m

system_state = [ system_stage v_init x_init y_init z_init balloon_diam_init gamma_init chi_init lat_init lon_init]
system_state_hist = system_state
wvector_hist = []
glider_control = [0 0]


## Parachute

Saber las dimensiones del parachute
Peso para detener la carga

¿Cómo deberminar la resistencia que debe ofrecer el paracaidas a la carga?


## Formas de hacer los cálculos:

### Vertical: 
zo = altura a la que se lanza


### horixontal:
xo =  posición horizontal en al que se encuentra


Aplicar las ecuaciones de Cinemática

In [ ]:
def temp(h_g):
    h_g = float(h_g)
    h = (6371000 * h_g) / (6371000 + h_g)
    if h <= 11000:
        t = 288 + -6.5e-3 * h
        
    elif 11000 < h <= 25000:
        t = 216.66
        
    elif 25000 < h <= 47000:
        t = 216.66 + 3e-3 * h
        
    elif 47000 < h <= 53000:
        t = 282.66
        
    elif 53000 < h <= 79000:
        t = 282.66 + -4.5e-3 * h
        
    elif 79000 < h <= 90000:
        t = 165.66
        
    elif 90000 < h <= 105000:
        t = 165.66 + 4e-3 * h
        
    else:
        print("You're out of the Troposphere")
        
    return t


def pressure(h_g):
    h_g = float(h_g)
    h = (6371000 * h_g) / (6371000 + h_g)
    
    if h <= 11000:              #TROPOSPHERE
        p = 101325 * ((288 + -6.5e-3 * h) / 288) ** (-9.81 / (287 * -6.5e-3))
        
    elif 11000 < h <= 25000:    #TROPOPAUSE
        p = 101325 * ((288 + -6.5e-3 * ((6371000 * 11000) / (6371000 + 11000))) / 288) ** (-9.81 / (287 * -6.5e-3))
        
    else:
        print("You're out of the Troposphere")
    
    return p


def density(h_g):
    h_g = float(h_g)
    h = (6371000 * h_g) / (6371000 + h_g)
    
    if h <= 11000:              #TROPOSPHERE
        rho = 1.225 * ((288 + -6.5e-3 * h) / 288) ** (-9.81 / (287 * -6.5e-3 - 1))
    
    elif 11000 < h <= 25000:    # TROPOPAUSE
        rho = 1.225 * ((288 + -6.5e-3 * ((6371000 * 11000) / (6371000 + 11000)))/ 288) ** (-9.81 / (287 * -6.5e-3 - 1))
    
    else:
        print("You're out of the Troposphere")
    
    return rho